<a href="https://colab.research.google.com/github/ruben216/realidadepublica/blob/main/Report_RP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Primeira extração: Emendas parlamentares
Antes de executar o código abaixo, é necessário possuir uma chave gerada no portal de transparência. Detalhes no Readme do repositório.


In [19]:
import pandas as pd
import requests
import json
import numpy as np
from google.colab import data_table
data_table.enable_dataframe_formatter()

# A biblioteca do Google Drive só é necessária para o caso de se exportar o dataframe
from google.colab import drive

#Comando para "criar" ou importar o seu Google Drive para dentro do Colab
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [4]:
# Chave gerada no portal
chave_portal = '6f98dcedb14a7c1053bc2bd8fb56ba5f'

# É necessário enviar a chave no Header da requisição
header_chave = {'chave-api-dados':chave_portal}

#Variavel que indica se tivemos conteúdo retornado do Portal
conteudo = 1

#Variável que controla a página que estamos iterando
pagina = 1

#Ano que iremos consultar
ano = 2021

#Objeto que irá armazenar os dados retornados de todas as páginas
dados_completos = []

# Laço que valida se o conteúdo retornado não está vazio.
while(conteudo > 0):

  # A URL é montada informando o ano e a página
  url = f'http://api.portaldatransparencia.gov.br/api-de-dados/emendas?ano={ano}&pagina={pagina}'
  
  # Neste ponto é enviada a request ao Portal de Transparencia, informando a URL e a chave.
  dados_pagina = requests.get(url, verify=True,headers=header_chave).json()
  
  # A variável *conteudo* recebe o tamanho do retorno recebido
  conteudo = len(dados_pagina)
  
  # Caso tenha conteúdo no retorno, ele é adicionado ao objeto de dados_completos
  if(conteudo > 0):
    dados_completos = dados_completos + dados_pagina
  pagina += 1
  
  #print(pagina)


df_dados_completos = pd.DataFrame(dados_completos)


In [20]:
#Caso queira visualizar os primeiros 5 registros, pode usar o head abaixo
df_dados_completos.head()

,codigoEmenda,ano,autor,nomeAutor,numeroEmenda,localidadeDoGasto,funcao,subfuncao,valorEmpenhado,valorLiquidado,valorPago,valorRestoInscrito,valorRestoCancelado,valorRestoPago,valorRestoAPagar
0,202081001576,2021,RELATOR GERAL,RELATOR GERAL,1576,Nacional,Urbanismo,Assistência comunitária,"0,00","0,00","0,00","9.000.000,00","0,00","0,00","9.000.000,00"
1,202129790005,2021,FABIO REIS,FABIO REIS,0005,Nacional,Direitos da cidadania,"Direitos individuais, coletivos e difusos","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
2,202139270010,2021,CAPITAO WAGNER,CAPITAO WAGNER,0010,CEARÁ,Direitos da cidadania,"Direitos individuais, coletivos e difusos","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
3,202136660003,2021,POMPEO DE MATTOS,POMPEO DE MATTOS,0003,RIO GRANDE DO SUL,Direitos da cidadania,"Direitos individuais, coletivos e difusos","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
4,202141340003,2021,VALDEVAN NOVENTA,VALDEVAN NOVENTA,0003,SERGIPE,Direitos da cidadania,"Direitos individuais, coletivos e difusos","0,00","0,00","0,00","0,00","0,00","0,00","0,00"


In [ ]:
# Para exportar o dataframe para o Google Drive
df_dados_completos.to_csv('/drive/MyDrive/Colab Notebooks/df_dados_completos.csv')

### Agrupando os dados por Estado

In [28]:

#df_dados_completos[df_dados_completos['localidadeDoGasto']=='SERGIPE']
df_dados_completos.groupby('localidadeDoGasto').count()['funcao'].sort_values(ascending=False).head(10)



localidadeDoGasto
SÃO PAULO            542
MINAS GERAIS         397
RIO DE JANEIRO       326
Nacional             292
RIO GRANDE DO SUL    287
BAHIA                221
PARANÁ               203
PERNAMBUCO           175
GOIÁS                154
SANTA CATARINA       145
Name: funcao, dtype: int64

### Agrupando os dados por parlamentar

In [29]:
df_dados_completos.groupby('autor').count()['funcao'].sort_values(ascending=False).head(10)

autor
RELATOR GERAL            34
OTTO ALENCAR             30
MARRECA FILHO            29
JUAREZ COSTA             27
MAURO BENEVIDES FILHO    26
JULIO CESAR              24
PROFESSORA MARCIVANIA    23
LUCAS REDECKER           23
ELI CORREA FILHO         22
ALAN RICK                22
Name: funcao, dtype: int64